## Setting Up:

In [9]:
import sys
import os
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
import zipfile
import cdsapi
import zipfile
import numpy as np
import glob
import xarray as xr

regions_path = '/home/mburlet/MBM/MassBalanceMachine/regions/French_Alps'
sys.path.insert(0, regions_path)
from scripts.config_FR import *

# Print paths to verify
print("Python path:", sys.path)
print("\nChecking if scripts directory exists:")
print(os.path.exists(os.path.join(regions_path, 'scripts')))
print("\nListing contents of regions directory:")
print(os.listdir(regions_path))

# Debug prints
print(f"1. Current working directory: {os.getcwd()}")
print(f"2. Regions path exists: {os.path.exists(regions_path)}")
print(f"3. Scripts directory exists: {os.path.exists(os.path.join(regions_path, 'scripts'))}")
print(f"4. Config file exists: {os.path.exists(os.path.join(regions_path, 'scripts', 'config_FR.py'))}")
print(f"5. sys.path: {sys.path}")

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

Python path: ['/home/mburlet/MBM/MassBalanceMachine/regions/French_Alps', '/home/mburlet/MBM/MassBalanceMachine/regions/French_Alps', '/home/mburlet/anaconda3/envs/MassBalanceMachine/lib/python311.zip', '/home/mburlet/anaconda3/envs/MassBalanceMachine/lib/python3.11', '/home/mburlet/anaconda3/envs/MassBalanceMachine/lib/python3.11/lib-dynload', '', '/home/mburlet/anaconda3/envs/MassBalanceMachine/lib/python3.11/site-packages', '/home/mburlet/MBM/MassBalanceMachine']

Checking if scripts directory exists:
True

Listing contents of regions directory:
['models', '1.3. ERA5Land-prepro.ipynb', '3.4. Feature selection.ipynb', '3.2. Train-ML-model.ipynb', '1.2. ERA5Land-prepro.ipynb', '1.1. GLAMOS-prepro.ipynb', 'prcp_fac_array.npy', '3.5. Train with GeoMB.ipynb', '1.2. OGGM-datapulling.ipynb', '1.1. GLACIOCLIM-prepro.ipynb', '3.3. Spatial-analysis.ipynb', 'scripts', 'w_prcp_array.npy']
1. Current working directory: /home/mburlet/MBM/MassBalanceMachine/regions/French_Alps
2. Regions path exis

## Download monthly ERA5-Land variables:

In [2]:
RUN = True
if RUN:
    os.makedirs(path_ERA5_raw, exist_ok=True)
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-land-monthly-means', {
            'product_type': ['monthly_averaged_reanalysis'],
            'variable': [
                '10m_u_component_of_wind',
                '10m_v_component_of_wind',
                '2m_temperature',
                'forecast_albedo',
                'snow_cover',
                'snow_density',
                'snow_depth_water_equivalent',
                'snowfall',
                'snowmelt',
                'surface_latent_heat_flux',
                'surface_net_thermal_radiation',
                'surface_sensible_heat_flux',
                'surface_solar_radiation_downwards',
                'total_precipitation',
            ],
            'year': [
                '1950',
                '1951',
                '1952',
                '1953',
                '1954',
                '1955',
                '1956',
                '1957',
                '1958',
                '1959',
                '1960',
                '1961',
                '1962',
                '1963',
                '1964',
                '1965',
                '1966',
                '1967',
                '1968',
                '1969',
                '1970',
                '1971',
                '1972',
                '1973',
                '1974',
                '1975',
                '1976',
                '1977',
                '1978',
                '1979',
                '1980',
                '1981',
                '1982',
                '1983',
                '1984',
                '1985',
                '1986',
                '1987',
                '1988',
                '1989',
                '1990',
                '1991',
                '1992',
                '1993',
                '1994',
                '1995',
                '1996',
                '1997',
                '1998',
                '1999',
                '2000',
                '2001',
                '2002',
                '2003',
                '2004',
                '2005',
                '2006',
                '2007',
                '2008',
                '2009',
                '2010',
                '2011',
                '2012',
                '2013',
                '2014',
                '2015',
                '2016',
                '2017',
                '2018',
                '2019',
                '2020',
                '2021',
                '2022',
                '2023',
                '2024',
            ],
            'month': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
            ],
            'time': ['00:00'],
            "data_format": "netcdf",
            "download_format": "zip",
            'area': [
                50,
                0,
                45,
                20,
            ],
        }, path_ERA5_raw+'download.netcdf.zip')
    with zipfile.ZipFile(path_ERA5_raw+'download.netcdf.zip', 'r') as zip:
        zip.extractall(path_ERA5_raw)
    c.retrieve("reanalysis-era5-single-levels", {
            "product_type": ["reanalysis"],
            "variable": ["geopotential"],
            "year": ["2024"],
            "month": ["06"],
            "day": ["01"],
            "time": ["12:00"],
            "data_format": "netcdf"
        }, path_ERA5_raw+'era5_geopotential_pressure.nc')

2025-05-11 10:21:49,127 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-11 10:21:49,128 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-11 10:21:49,265 INFO Request ID is 0e1df900-7c18-4c6d-8a86-a97438a12b59
2025-05-11 10:21:49,326 INFO status has been updated to accepted
2025-05-11 10:22:02,843 INFO status has been updated to running
2025-05-11 10:22:10,473 INFO status has been updated to successful


6047379c0db16d8b57d8f328cf963b2f.zip:   0%|          | 0.00/262M [00:00<?, ?B/s]

2025-05-11 10:22:13,925 INFO Request ID is 61d70879-33aa-404c-af0e-21021ebd2438
2025-05-11 10:22:13,960 INFO status has been updated to accepted
2025-05-11 10:22:27,342 INFO status has been updated to successful


aa9d83a880f1bb02cf32b143dfe1d3fa.nc:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

In [ ]:
"""
import cdsapi

dataset = "reanalysis-era5-land-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": [
        "2m_temperature",
        "snow_cover",
        "snow_density",
        "snow_depth_water_equivalent",
        "snowfall",
        "snowmelt",
        "forecast_albedo",
        "surface_latent_heat_flux",
        "surface_net_thermal_radiation",
        "surface_sensible_heat_flux",
        "surface_solar_radiation_downwards",
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "total_precipitation"
    ],
    "year": [
        "1957", "1958", "1959",
        "1960", "1961", "1962",
        "1963", "1964", "1965",
        "1966", "1967", "1968",
        "1969", "1970", "1971",
        "1972", "1973", "1974",
        "1975", "1976", "1977",
        "1978", "1979", "1980",
        "1981", "1982", "1983",
        "1984", "1985", "1986",
        "1987", "1988", "1989",
        "1990", "1991", "1992",
        "1993", "1994", "1995",
        "1996", "1997", "1998",
        "1999", "2000", "2001",
        "2002", "2003", "2004",
        "2005", "2006", "2007",
        "2008", "2009", "2010",
        "2011", "2012", "2013",
        "2014", "2015", "2016",
        "2017", "2018", "2019",
        "2020", "2021", "2022"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "zip",
    "area": [47, 3, 43, 8]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": ["geopotential"],
    "year": ["2024"],
    "month": ["06"],
    "day": ["01"],
    "time": ["12:00"],
    "data_format": "netcdf"
}

client = cdsapi.Client()
client.retrieve(dataset, request, "era5_geopotential_pressure.nc")
"""

HTTPError: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-land-monthly-means


In [3]:
! ls '/home/mburlet/scratch/data/DATA_MB/ERA5Land/raw/'

data_stream-moda.nc	       era5_geopotential_pressure_scratch.nc
download.netcdf.zip	       era5_monthly_averaged_data_scratch.nc
era5_geopotential_pressure.nc


In [4]:
dc=xr.open_dataset(path_ERA5_raw+'data_stream-moda.nc')

In [5]:
dc2 = dc.rename({'valid_time': 'time'}) # Coordinates have changed recently in the API, this is to keep compatibility with our code

In [6]:
dc2.to_netcdf(path_ERA5_raw + "era5_monthly_averaged_data.nc")

In [3]:
display(xr.open_dataset(path_ERA5_raw+'era5_geopotential_pressure.nc'))

display(xr.open_dataset(path_ERA5_raw+'era5_geopotential_pressure_scratch.nc'))

<xarray.Dataset> Size: 4MB
Dimensions:     (valid_time: 1, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 8B 2024-06-01T12:00:00
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver      <U4 16B ...
Data variables:
    z           (valid_time, latitude, longitude) float32 4MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T19:59 GRIB to CDM+CF via cfgrib-0.9.1...

<xarray.Dataset> Size: 52MB
Dimensions:    (longitude: 3600, latitude: 1801, time: 1)
Coordinates:
  * longitude  (longitude) float32 14kB 0.0 0.1 0.2 0.3 ... 359.7 359.8 359.9
  * latitude   (latitude) float32 7kB 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 8B 2013-08-09T12:00:00
Data variables:
    z          (time, latitude, longitude) float64 52MB ...
Attributes:
    Conventions:               CF-1.6
    history:                   Fri Jun 12 14:41:56 2020: ncpdq -U geo_1279l4_...
    NCO:                       4.7.2
    nco_openmp_thread_number:  1

In [2]:
# Load both datasets
new_geopotential = xr.open_dataset(path_ERA5_raw+'era5_geopotential_pressure.nc')
old_geopotential = xr.open_dataset(path_ERA5_raw+'era5_geopotential_pressure_scratch.nc')

# 1. Compare basic structure and metadata
print("=== NEW API DATASET ===")
print(new_geopotential.info())
print("\nAttributes:", new_geopotential.attrs)
print("\nVariables:", list(new_geopotential.variables))

print("\n=== OLD API DATASET ===")
print(old_geopotential.info())
print("\nAttributes:", old_geopotential.attrs)
print("\nVariables:", list(old_geopotential.variables))

# 2. Compare the geopotential variable directly
if 'z' in new_geopotential and 'z' in old_geopotential:
    # Check a sample point (adjust coordinates as needed)
    lat, lon = 45.0, 7.0  # Example coordinates in the French Alps
    
    new_z = new_geopotential.z.sel(latitude=lat, longitude=lon, method='nearest')
    old_z = old_geopotential.z.sel(latitude=lat, longitude=lon, method='nearest')
    
    print(f"\nGeopotential at ({lat}, {lon}):")
    print(f"New API: {new_z.values}")
    print(f"Old API: {old_z.values}")
    
    # Calculate altitude using your formula for both
    r_earth = 6367.47 * 10e3  # [m] 
    g = 9.80665  # [m/s^2]
    
    new_alt = r_earth * (new_z.values / g) / (r_earth - (new_z.values / g))
    old_alt = r_earth * (old_z.values / g) / (r_earth - (old_z.values / g))
    
    print(f"\nCalculated ALTITUDE_CLIMATE:")
    print(f"New API: {new_alt}")
    print(f"Old API: {old_alt}")
    print(f"Difference: {new_alt - old_alt} meters")

=== NEW API DATASET ===
xarray.Dataset {
dimensions:
	valid_time = 1 ;
	latitude = 721 ;
	longitude = 1440 ;

variables:
	int64 number() ;
		number:long_name = ensemble member numerical id ;
		number:units = 1 ;
		number:standard_name = realization ;
	datetime64[ns] valid_time(valid_time) ;
		valid_time:long_name = time ;
		valid_time:standard_name = time ;
	float64 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:standard_name = latitude ;
		latitude:long_name = latitude ;
		latitude:stored_direction = decreasing ;
	float64 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:standard_name = longitude ;
		longitude:long_name = longitude ;
	<U4 expver() ;
	float32 z(valid_time, latitude, longitude) ;
		z:GRIB_paramId = 129 ;
		z:GRIB_dataType = an ;
		z:GRIB_numberOfPoints = 1038240 ;
		z:GRIB_typeOfLevel = surface ;
		z:GRIB_stepUnits = 1 ;
		z:GRIB_stepType = instant ;
		z:GRIB_gridType = regular_ll ;
		z:GRIB_uvRelativeToGrid = 0 ;
		z:GRIB_NV = 0 

In [10]:
display(pd.read_csv('/home/mburlet/scratch/data/DATA_MB/GLACIOCLIM/WGMS_all/FR_wgms_dataset_monthly_full.csv').head(2))
display(pd.read_csv('/home/mburlet/scratch/data/DATA_MB/GLACIOCLIM/WGMS_all/FR_wgms_dataset_monthly_full_oldAPI.csv').head(2))
display(pd.read_csv('/home/mburlet/scratch/data/DATA_MB/GLACIOCLIM/WGMS_all/FR_wgms_dataset_monthly_full_mixedAPI.csv').head(2))
display(pd.read_csv('/home/mburlet/scratch/data/DATA_MB/GLACIOCLIM/WGMS_all/FR_wgms_dataset_monthly_full_allnewmarijnAPI.csv').head(2))

,YEAR,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,POINT_ELEVATION,RGIId,POINT_ID,ID,...,millan_v,t2m,tp,slhf,sshf,ssrd,fal,str,u10,v10
0,2006,7.024819,45.944607,-0.67,1910.456055,1123.063945,3033.52,RGI60-11.03638,Argentiere_Tour_Noir_annual_smb_abl_2006_setup...,0,...,5.479116,1.863434,0.002194,-1039166.0,-437704.0,10756431.0,0.497862,-3961346.0,-0.321241,0.331826
1,2006,7.024819,45.944607,-0.67,1910.456055,1123.063945,3033.52,RGI60-11.03638,Argentiere_Tour_Noir_annual_smb_abl_2006_setup...,0,...,5.479116,-5.253510,0.001675,-277848.0,-212654.0,7246084.0,0.564720,-3621156.0,-0.175529,0.153369


,YEAR,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,POINT_ELEVATION,RGIId,POINT_ID,ID,...,millan_v,t2m,tp,slhf,sshf,ssrd,fal,str,u10,v10
0,2006,7.024819,45.944607,-0.67,2524.444781,509.075219,3033.52,RGI60-11.03638,Argentiere_Tour_Noir_annual_smb_abl_2006_setup...,0,...,5.479116,1.863512,0.002194,-1.039061e+06,-437717.194726,1.075649e+07,0.497858,-3.961348e+06,-0.321163,0.331876
1,2006,7.024819,45.944607,-0.67,2524.444781,509.075219,3033.52,RGI60-11.03638,Argentiere_Tour_Noir_annual_smb_abl_2006_setup...,0,...,5.479116,-5.253318,0.001675,-2.778184e+05,-212737.035722,7.246017e+06,0.564715,-3.621123e+06,-0.175581,0.153386


,YEAR,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,POINT_ELEVATION,RGIId,POINT_ID,ID,...,millan_v,t2m,tp,slhf,sshf,ssrd,fal,str,u10,v10
0,2006,7.024819,45.944607,-0.67,2524.444781,509.075219,3033.52,RGI60-11.03638,Argentiere_Tour_Noir_annual_smb_abl_2006_setup...,0,...,5.479116,1.863434,0.002194,-1039166.0,-437704.0,10756431.0,0.497862,-3961346.0,-0.321241,0.331826
1,2006,7.024819,45.944607,-0.67,2524.444781,509.075219,3033.52,RGI60-11.03638,Argentiere_Tour_Noir_annual_smb_abl_2006_setup...,0,...,5.479116,-5.253510,0.001675,-277848.0,-212654.0,7246084.0,0.564720,-3621156.0,-0.175529,0.153369


,YEAR,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,POINT_ELEVATION,RGIId,POINT_ID,ID,...,millan_v,t2m,tp,slhf,sshf,ssrd,fal,str,u10,v10
0,2006,7.024819,45.944607,-0.67,2524.444781,509.075219,3033.52,RGI60-11.03638,Argentiere_Tour_Noir_annual_smb_abl_2006_setup...,0,...,5.479116,1.863434,0.002194,-1039166.0,-437704.0,10756431.0,0.497862,-3961346.0,-0.321241,0.331826
1,2006,7.024819,45.944607,-0.67,2524.444781,509.075219,3033.52,RGI60-11.03638,Argentiere_Tour_Noir_annual_smb_abl_2006_setup...,0,...,5.479116,-5.253510,0.001675,-277848.0,-212654.0,7246084.0,0.564720,-3621156.0,-0.175529,0.153369
